In [43]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import BooleanAdjective, PointerAdjective, NodesGroupPointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, If, ConditionalExplanation, CompositeExplanation

## Example search tree:

In [44]:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
leaf31 = MinMaxNode('leaf21', score=1)
leaf32 = MinMaxNode('leaf22', score=1)

child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
child3 = MinMaxNode('child3', children=[leaf31, leaf32], score_child=leaf31)

root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2, child3], score_child=child1)

## Framework definition

In [45]:
knowledgebase = ArgumentationFramework()

knowledgebase.add_adjectives([
    
    BooleanAdjective("leaf",
        definition = "node.is_leaf"),


    PointerAdjective("score",
        definition = "node.score",

        explanation = ConditionalExplanation(
            condition = If("leaf"),
            explanation_if_true = Assumption("Leaf nodes have scores from the evaluation function"),
            explanation_if_false = CompositeExplanation(
                Assumption("Internal nodes have scores from children"),
                Possession("backtracing child"))
        )),


    BooleanAdjective("opponent player turn",
        definition = "not node.maximizing_player_turn"),


    PointerAdjective("backtracing child",
        definition = "node.score_child",
        # alternative definition for backtracing child:
        # min(node.children, key=lambda child: child.score),

        explanation = ConditionalExplanation(
            condition = If("opponent player turn"),
            explanation_if_true = CompositeExplanation(
                Assumption("We assume the opponent will do their best move."),
                Possession("backtracing child", "worst")),
            explanation_if_false = CompositeExplanation(
                Assumption("On our turn we take the maximum rated move."),
                Possession("backtracing child", "best"))
        )),

    ComparisonAdjective("better", "score", ">"),
 
    NodesGroupPointerAdjective("siblings",
        definition = "node.parent.children",
        excluding = "node"),

    MaxRankAdjective("best", "better", "siblings"),

    MinRankAdjective("worst", "better", "siblings"),
])

In [46]:
print(knowledgebase)

Propositions:
node is leaf
node has score = ?
node is opponent player turn
node has backtracing child = ?
node1 is better than node2
node has siblings = ?
node is best
node is worst


Implications:
[node is leaf ←
 (assumption) Definition of "leaf" is "node.is_leaf"]
[node has score = ? ←
 [node is leaf ←
 (assumption) Definition of "leaf" is "node.is_leaf"]
 ∧ (assumption) Leaf nodes have scores from the evaluation function
 || [¬(node is leaf) ←
 (assumption) Definition of "leaf" is "node.is_leaf"]
 ∧ (assumption) Internal nodes have scores from children
 ∧ [node has backtracing child = ? ←
 [node is opponent player turn ←
 (assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn"]
 ∧ (assumption) We assume the opponent will do their best move.
 ∧ [node is worst ←
 (assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings" ∧ Node ¬(better) than all nodes in siblings]
 || [¬(node is opponent player turn) ←
 (ass

In [47]:
explainer = ArgumentativeExplainer(knowledgebase)

## Set your explanations settings

In [48]:
explainer.set_tree_search_motivation(lambda root: root.children, "best")

In [49]:
from src.explainer.explanation_tactics import OnlyRelevantComparisons
explainer.add_explanation_tactic("best", OnlyRelevantComparisons(mode = "top_3"))
explainer.del_explanation_tactic("best", "OnlyRelevantComparisons")



KeyboardInterrupt



In [ ]:
settings = {
            'explanation_depth': 5 ,
            'assumptions_verbosity': 'verbose'
        }

explainer.configure_settings(settings)

## You can now explain nodes' properties

In [ ]:
explainer.explain_adjective(child1, "best")

[child1 is best ←
 	(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings"
	 ∧ [child1 has siblings = child2, child3 ←
	 		(assumption) Definition of "siblings" is "node.parent.children excluding node"]
	 ∧ [child1 is better than child2 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ [child1 has score = 3 ←
			 			[¬(child1 is leaf) ←
						 				(assumption) Definition of "leaf" is "node.is_leaf"]
						 ∧ (assumption) Internal nodes have scores from children
						 ∧ [child1 has backtracing child = leaf11 ←
						 				[child1 is opponent player turn ←
										 					(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn"]
										 ∧ (assumption) We assume the opponent will do their best move.
										 ∧ [leaf11 is worst ←
										 					(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings"
															 

In [ ]:
explainer.add_explanation_tactic("best", OnlyRelevantComparisons(mode = "top_1"))

In [ ]:
explainer.explain_adjective(child1, "best")

In [ ]:
settings = {
            'explanation_depth': 2 ,
            'assumptions_verbosity': 'verbose'
        }

explainer.configure_settings(settings)

In [ ]:
explainer.explain_adjective(child1, "worst", explanation_depth = 0)

¬(child1 is worst)


In [ ]:
explainer.explain_adjective(child1, "worst", explanation_depth = 1)

[¬(child1 is worst) ←
 	(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings"
	 ∧ child1 has siblings = child2, child3
	 ∧ child1 is better than child2 ∧ child1 is better than child3]


In [ ]:
explainer.explain_adjective(child1, "worst")

[¬(child1 is worst) ←
 	(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings"
	 ∧ [child1 has siblings = child2, child3 ←
	 		(assumption) Definition of "siblings" is "node.parent.children excluding node"]
	 ∧ [child1 is better than child2 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ child1 has score = 3 ∧ child2 has score = 2]
	 ∧ [child1 is better than child3 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ child1 has score = 3 ∧ child3 has score = 1]]


In [ ]:
explainer.explain_adjective(child1, "worst", explanation_depth = 3, print_depth=True)

[¬(child1 is worst) ←
 	Depth 1:
	(assumption) By definition a node is "worst" if it's ¬("better") compared to all nodes among "siblings"
	 ∧ [child1 has siblings = child2, child3 ←
	 		Depth 2:
			(assumption) Definition of "siblings" is "node.parent.children excluding node"]
	 ∧ [child1 is better than child2 ←
	 		Depth 2:
			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ [child1 has score = 3 ←
			 			Depth 3:
						¬(child1 is leaf)
						 ∧ (assumption) Internal nodes have scores from children ∧ child1 has backtracing child = leaf11]
			 ∧ [child2 has score = 2 ←
			 			Depth 3:
						¬(child2 is leaf)
						 ∧ (assumption) Internal nodes have scores from children ∧ child2 has backtracing child = leaf22]]
	 ∧ [child1 is better than child3 ←
	 		Depth 2:
			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ [child1 has score = 3 ←
			 			Depth 3:
						¬(child1 is leaf)
						 ∧ (assumption) Inter

In [ ]:
explainer.explain_adjective(child1, "better", child2)

[child1 is better than child2 ←
 	(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
	 ∧ [child1 has score = 3 ←
	 		¬(child1 is leaf)
			 ∧ (assumption) Internal nodes have scores from children ∧ child1 has backtracing child = leaf11]
	 ∧ [child2 has score = 2 ←
	 		¬(child2 is leaf)
			 ∧ (assumption) Internal nodes have scores from children ∧ child2 has backtracing child = leaf22]]


In [ ]:
explainer.explain_adjective(child1, "siblings")

[child1 has siblings = child2, child3 ←
 	(assumption) Definition of "siblings" is "node.parent.children excluding node"]


In [ ]:
explainer.explain_adjective(child1, "best")

[child1 is best ←
 	(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings"
	 ∧ [child1 has siblings = child2, child3 ←
	 		(assumption) Definition of "siblings" is "node.parent.children excluding node"]
	 ∧ [child1 is better than child2 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ child1 has score = 3 ∧ child2 has score = 2]
	 ∧ [child1 is better than child3 ←
	 		(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ child1 has score = 3 ∧ child3 has score = 1]]


In [ ]:
explainer.explain_adjective(root, "backtracing child")

[root has backtracing child = child1 ←
 	[¬(root is opponent player turn) ←
	 		(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn"]
	 ∧ (assumption) On our turn we take the maximum rated move.
	 ∧ [child1 is best ←
	 		(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings"
			 ∧ child1 has siblings = child2, child3
			 ∧ child1 is better than child2 ∧ child1 is better than child3]]


In [ ]:
explainer.explain_adjective(child1, "best", explanation_depth=6, print_depth=True)

[child1 is best ←
 	Depth 1:
	(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings"
	 ∧ [child1 has siblings = child2, child3 ←
	 		Depth 2:
			(assumption) Definition of "siblings" is "node.parent.children excluding node"]
	 ∧ [child1 is better than child2 ←
	 		Depth 2:
			(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
			 ∧ [child1 has score = 3 ←
			 			Depth 3:
						[¬(child1 is leaf) ←
						 				Depth 4:
										(assumption) Definition of "leaf" is "node.is_leaf"]
						 ∧ (assumption) Internal nodes have scores from children
						 ∧ [child1 has backtracing child = leaf11 ←
						 				Depth 4:
										[child1 is opponent player turn ←
										 					Depth 5:
															(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn"]
										 ∧ (assumption) We assume the opponent will do their best move.
										 ∧ [leaf11 is worst ←
										 					Depth 5:


In [ ]:
explainer.explain_adjective(root, "score", explanation_depth = 15, print_depth=True)

[root has score = 3 ←
 	Depth 1:
	[¬(root is leaf) ←
	 		Depth 2:
			(assumption) Definition of "leaf" is "node.is_leaf"]
	 ∧ (assumption) Internal nodes have scores from children
	 ∧ [root has backtracing child = child1 ←
	 		Depth 2:
			[¬(root is opponent player turn) ←
			 			Depth 3:
						(assumption) Definition of "opponent player turn" is "not node.maximizing_player_turn"]
			 ∧ (assumption) On our turn we take the maximum rated move.
			 ∧ [child1 is best ←
			 			Depth 3:
						(assumption) By definition a node is "best" if it's "better" compared to all nodes among "siblings"
						 ∧ [child1 has siblings = child2, child3 ←
						 				Depth 4:
										(assumption) Definition of "siblings" is "node.parent.children excluding node"]
						 ∧ [child1 is better than child2 ←
						 				Depth 4:
										(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score
										 ∧ [child1 has score = 3 ←
										 					Depth 5:
															[¬(child1 is l

Track down that Not before the "Considering your definition of leaf"

In [ ]:
explainer.query_explanation(root, "Why is child 1 maxoptimal?")